In [ ]:
!pip install dgl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 10.2 MB/s eta 0:00:00
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


### Dependencies

In [ ]:
import torch
import dgl
from collections import namedtuple
import dgl.function as fn
from copy import deepcopy as dc
import random
import time
from time import time
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt


## Environment

In [ ]:
import torch
import dgl
from collections import namedtuple
import dgl.function as fn
from copy import deepcopy as dc
import random
import time
from time import time
from torch.utils.data import DataLoader
import numpy as np
from collections import deque

class IM(object):
    def __init__(self, max_budget, p, num_nodes, cost):
        self.max_budget = max_budget
        self.BUDGET = max_budget
        assert(p <= 1 and p >= 0)
        self.p = p
        self.num_nodes = num_nodes
        self.cost = cost

    def compute_reward(self, state):
        # reward is the number of additional nodes influenced
        reward = 0
        # each node has one chance to influence each neighbour
        # print(state)
        new_influenced = state.detach().cpu().numpy().ravel()
        # print(new_influenced)
        tot_influenced = state.detach().cpu().numpy().ravel()
        while((new_influenced == 1).sum() >= 1):
            # next = torch.full(
            #     (self.num_nodes, 1),
            #     0, 
            #     dtype = torch.long
            #     )
            next = np.zeros(self.num_nodes)
            for e in range(self.g.number_of_edges()):
                # print(new_influenced[self.g.edges()[0][e]])
                if((new_influenced[self.g.edges()[0][e]] == 1) and 
                   not(tot_influenced[self.g.edges()[1][e]] == 1 or new_influenced[self.g.edges()[1][e]] == 1)):
                    r = random.random()
                    if(r < self.p):
                        # node influenced
                        next[self.g.edges()[1][e]] = 1
                        reward += 1
            # print(new_influenced)
            tot_influenced = tot_influenced + new_influenced
            new_influenced = next
        return reward
     
    def step(self, action):
        reward, sol, done = self._take_action(action)
        
        ob = self._build_ob()
        self.sol = sol
        info = {"sol": self.sol}

        # need to convert ob to ndarray from tensor
        ob = ob.detach().cpu().numpy().ravel()
        next_state = np.copy(ob)
        return next_state, reward, done, info
    
    def _take_action(self, action):
        r1, r2 = 0, 0
        num_iter = 100
        for i in range(num_iter):
            r1 += self.compute_reward(self.x[:-1])
        if(self.x[action] == 0 and self.cost[action] <= self.max_budget):
            self.x[action] = 1
            self.x[-1] -= self.cost[action]
            self.max_budget -= self.cost[action]
        # write code for else case 
        next_sol = 0
        for i in range(num_iter):
            r2 += self.compute_reward(self.x[:-1])
        done = self._check_done()
        return (r2 - r1)/num_iter, next_sol, done

    def _check_done(self): 
        inactive = (self.x[:-1] == 0).type(torch.float)
        # print(inactive)
        self.g.ndata['h'] = inactive
        not_selected = dgl.sum_nodes(self.g, 'h')
        self.g.ndata.pop('h')
        done = (not_selected == 0) or (self.max_budget <= 0)
        return done
                
    def _build_ob(self):
        ob_x = self.x
        # ob = torch.cat([ob_x], dim = 2)
        # return ob
        return ob_x
    
    # using num_samples = 1 as of now 
    def register(self, g, num_samples = 1):
        self.g = g
        self.g.set_n_initializer(dgl.init.zero_initializer)
        t = torch.full((self.num_nodes, 1), 0, dtype=torch.float16)
        # torch.full(
            #     (self.num_nodes, 1),
            #     0, 
            #     dtype = torch.long
            #     )
        self.x = torch.cat((t, torch.tensor([[self.max_budget]])), 0)
        ob = self._build_ob()
        return ob


    def reset(self):
        state = np.zeros(self.num_nodes + 1)
        state[-1] = self.BUDGET

        t = torch.full((self.num_nodes, 1), 0, dtype=torch.float16)
        self.max_budget = self.BUDGET
        self.x = torch.cat((t, torch.tensor([[self.max_budget]])), 0)

        return np.array(state)

## Main 

In [ ]:
cost = torch.tensor([300, 300, 300, 300, 300])
maxb = 1000
env = IM(maxb, 0.6, 5, cost)
src_ids = torch.tensor([0, 1, 2, 3])
dst_ids = torch.tensor([1, 2, 3, 4])
g = dgl.graph((src_ids, dst_ids), num_nodes=5)
ob = env.register(g)


print(ob)



tensor([[   0.],
        [   0.],
        [   0.],
        [   0.],
        [   0.],
        [1000.]], dtype=torch.float16)


### greedy

At each step pick the node with highest influence per cost

In [ ]:
import copy

cost = torch.tensor([300, 300, 300, 300, 300])
maxb = 1000
env = IM(maxb, 0.6, 5, cost)
src_ids = torch.tensor([0, 1, 2, 3])
dst_ids = torch.tensor([1, 2, 3, 4])
g = dgl.graph((src_ids, dst_ids), num_nodes=5)
ob = env.register(g)

def greedy():
    score = 0
    n = env.num_nodes
    while True:
        reward_per_cost = np.zeros(n)

        done = False
        for i in range(n):
            temp = copy.deepcopy(env)
            if env.x[i] == 0 and env.cost[i] <= env.max_budget :
                _, reward, _, _ = temp.step(i)
                reward_per_cost[i] = reward / temp.cost[i]
            
        choice = np.argmax(reward_per_cost)
        if reward_per_cost[choice] == 0:
            break

        print(" reward_per_cost : ", reward_per_cost)
        print("choice : ", choice)
        
        _, reward, done, _ = env.step(choice)
        print(env.x, env.max_budget)
        score += reward

        if done == True:
            break

    return score


score = greedy()
print(score)

 reward_per_cost :  [0.0037     0.00463333 0.00333333 0.00206667 0.        ]
choice :  1
tensor([[  0.],
        [  1.],
        [  0.],
        [  0.],
        [  0.],
        [700.]], dtype=torch.float16) tensor(700)
 reward_per_cost :  [ 0.00036667  0.         -0.0008      0.00216667 -0.00056667]
choice :  3
tensor([[  0.],
        [  1.],
        [  0.],
        [  1.],
        [  0.],
        [400.]], dtype=torch.float16) tensor(400)
1.56
